# Data loading 
Next cells are used to load the data we need.

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1 
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import sys
import matplotlib.pyplot as plt
sys.path.append('../../')


In [2]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('../data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [3]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [4]:
ICM_df = pd.read_csv('../data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [5]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [6]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Base.Evaluation.Evaluator import EvaluatorHoldout

URM_train_1, URM_validation_1 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=1)

URM_train_2, URM_validation_2 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=2)

URM_train_3, URM_validation_3 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=3)

URM_train_4, URM_validation_4 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=4)

URM_train_5, URM_validation_5 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=5)

training_list = [URM_train_1,URM_train_2,URM_train_3,URM_train_4,URM_train_5]

evaluator_validation_1 = EvaluatorHoldout(URM_validation_1, cutoff_list=[10])
evaluator_validation_2 = EvaluatorHoldout(URM_validation_2, cutoff_list=[10])
evaluator_validation_3 = EvaluatorHoldout(URM_validation_3, cutoff_list=[10])
evaluator_validation_4 = EvaluatorHoldout(URM_validation_4, cutoff_list=[10])
evaluator_validation_5 = EvaluatorHoldout(URM_validation_5, cutoff_list=[10])

evaluator_list = [evaluator_validation_1, evaluator_validation_2, evaluator_validation_3, evaluator_validation_4, evaluator_validation_5]


In [11]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
i=1
for URM_train, evaluator in zip(training_list,evaluator_list):
    URM_train_ICM_all_SLIM = sps.vstack([URM_train*(1- 0.6229636472829045), ICM_all.T* 0.6229636472829045])

    recommender_SLIM_BPR = SLIM_BPR_Cython(URM_train_ICM_all_SLIM, recompile_cython=False, verbose=False)
    recommender_SLIM_BPR.fit(epochs=87, sgd_mode= "adam", learning_rate= 6.12252603066224e-06, topK = 805, batch_size = 35, lambda_i=3.0399585540912118e-05, lambda_j=0.00012176512055401498 )
    #recommender.URM_train = URM_train.tocsr()
    
    result_dict, _ = evaluator.evaluateRecommender(recommender_SLIM_BPR)
    print(result_dict)
    
    filename = 'SLIM_BPR_fold_number' + str(i)
    recommender_SLIM_BPR.save_model('SLIM_BPR_saved/', file_name = filename)
    i = i+1

Deallocating Cython objects
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 87. Elapsed time 0.09 sec
SLIM_BPR_Recommender: Epoch 2 of 87. Elapsed time 0.17 sec
SLIM_BPR_Recommender: Epoch 3 of 87. Elapsed time 0.25 sec
SLIM_BPR_Recommender: Epoch 4 of 87. Elapsed time 0.33 sec
SLIM_BPR_Recommender: Epoch 5 of 87. Elapsed time 0.41 sec
SLIM_BPR_Recommender: Epoch 6 of 87. Elapsed time 0.50 sec
SLIM_BPR_Recommender: Epoch 7 of 87. Elapsed time 0.58 sec
SLIM_BPR_Recommender: Epoch 8 of 87. Elapsed time 0.66 sec
SLIM_BPR_Recommender: Epoch 9 of 87. Elapsed time 0.74 sec
SLIM_BPR_Recommender: Epoch 10 of 87. Elapsed time 0.82 sec
SLIM_BPR_Recommender: Epoch 11 of 87. Elapsed time 0.90 sec
SLIM_BPR_Recommender: Epoch 12 of 87. Elapsed time 0.98 sec
SLIM_BPR_Recommender: Epoch 13 of 87. Elapsed time 1.06 sec
SLIM_BPR_Recommender: Epoch 14 of 87. Elapsed time 1.14 sec
SLIM_BPR_Recommender: Epoch 15 of 87. Elapsed time 1.22 sec
SLIM_BPR_Recommender: Epoch